In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/My\ Drive/DL_assignments/A4/machine_translation/
 
import torch
import pickle
import numpy as np
from PIL import Image
from torchvision import transforms
import re
import torch.nn as nn
import torch.nn.functional as F
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method1

/content/drive/My Drive/DL_assignments/A4/machine_translation


In [ ]:
# Loading data
 
# english_words = np.load("english_words_without.npy",allow_pickle=True).tolist() # english vocabulary
# tamil_words = np.load("tamil_words_without.npy",allow_pickle= True).tolist() # tamil vocabulary
# initial_embeddings = np.load("initial_en_embeddings_without.npy",allow_pickle=True) # initial glove embeddings for english words 
# english_sentences = np.load("english_sentences_without_punctuation.npy",allow_pickle=True) # English sentences
# tamil_sentences = np.load("tamil_sentences_without_punctuation.npy",allow_pickle=True) # Tamil sentences

english_words = np.load("english_words_without.npy",allow_pickle=True).tolist() # english vocabulary
tamil_words = np.load("tamil_words_without.npy",allow_pickle= True).tolist() # tamil vocabulary
initial_embeddings = np.load("initial_embeddings_without.npy",allow_pickle=True) # initial glove embeddings for english words 
english_sentences_dev = np.load("english_sentences_without_punctuation_test.npy",allow_pickle=True) # English sentences
tamil_sentences_dev = np.load("tamil_sentences_without_punctuation_test.npy",allow_pickle=True) # Tamil sentences

In [ ]:
english_sentences_dev = np.load("english_sentences_without_punctuation_dev.npy",allow_pickle=True)
tamil_sentences_dev = np.load("tamil_sentences_without_punctuation_dev.npy",allow_pickle=True)

In [ ]:
english_sentences_dev = np.load("english_sentences_shuffled_dev.npy",allow_pickle=True)
tamil_sentences_dev = np.load("tamil_sentences_shuffled_dev.npy",allow_pickle=True)

In [ ]:
# RNN Encoder Decoder
 
class Network(nn.Module):
    def create_emb_layer(self,weights_matrix,flag):
        num_embeddings, embedding_dim = weights_matrix.size()
        if flag == True:
            emb_layer = nn.Embedding(num_embeddings, embedding_dim)
            emb_layer.load_state_dict({'weight': weights_matrix})
        else:
            emb_layer = nn.Embedding(len(tamil_words), embedding_dim)
        emb_layer.weight.requires_grad = True
        return emb_layer
        
    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def __init__(self):
        super().__init__()
        
        self.num_encoder_layers = 2
        self.num_decoder_layers = 2
        self.num_attention_heads = 4
        
        self.en_embeddings = self.create_emb_layer(torch.from_numpy(initial_embeddings),True)
        self.ta_embeddings = self.create_emb_layer(torch.from_numpy(initial_embeddings),False)
        
        self.transformer = nn.Transformer(dim_feedforward=300,d_model=300, nhead=self.num_attention_heads, num_encoder_layers=self.num_encoder_layers,num_decoder_layers=self.num_decoder_layers)
        self.linear_layer = nn.Sequential(
            nn.Linear(300, 300)
        )
        self.output_layer = nn.Sequential(
            nn.Linear(300, len(tamil_words))
        )
 
    
 
    def forward(self, x,y):
        mask = self._generate_square_subsequent_mask(y.size(0))
        
        '''
        output = self.transformer(torch.unsqueeze(self.en_embeddings(x),1),torch.unsqueeze(self.ta_embeddings(y),1),tgt_mask=mask)
        output = output.squeeze(1)
        outs = self.linear_layer(output)
        return self.output_layer(outs)
        '''
        out = []
        out_i = []
        first = True
        while True:
            if len(out) == 40:
              return out,out_i 
            if not first:
                o = torch.squeeze(output[-1,:,:],1)
                o = self.linear_layer(o)
                o = self.output_layer(o)
                v,i = torch.max(o,1)
                i = i.item()
                if i == len(tamil_words) - 1:
                    break
                elif i == len(tamil_words) - 2:
                    out.append("st")
                    out_i.append(i)
                else:
                    out.append(tamil_words[i])
                    out_i.append(i)
            else:
                i = len(tamil_words) - 2
                first = False
                out_i.append(i)
                output = self.transformer(torch.unsqueeze(self.en_embeddings(x),1),torch.unsqueeze(self.ta_embeddings(torch.tensor(out_i)),1))
                
            if i != len(tamil_words) - 1:
                
                output = self.transformer(torch.unsqueeze(self.en_embeddings(x),1),torch.unsqueeze(self.ta_embeddings(torch.tensor(out_i)),1))
            else:
                break
        return out,out_i 
        #'''
        

In [ ]:
# Training 
 
# dev = torch.device("cpu")
 
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu" 
print("using {}".format(dev))
 
dev = "cpu"
model = Network().to(dev)
criterion = nn.CrossEntropyLoss().to(dev)
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.01)

start_epoch = 45
num_epochs = 100 
train_from_saved_model = True
 
if train_from_saved_model:
    checkpoint = torch.load("model/model_translation_transformer_without_withmask_{}.pth".format(start_epoch),map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
 

for e in range(start_epoch + 1,start_epoch + num_epochs + 1):
    print(e)
    epoch_loss = 0
    optimizer.zero_grad()
    batch_size = 1
    d = 0
    for it in range(int(25500/batch_size)):
        loss = 0
        optimizer.zero_grad()
        expected = english_sentences[it]
        expected = [len(english_words) - 2] + [english_words.index(s.lower()) for s in expected]+ [len(english_words) - 1]
        expected_in = np.array(expected[1:-1])
        expected = tamil_sentences[it]
        expected = [len(tamil_words) - 2] + [tamil_words.index(s.lower()) for s in expected]+ [len(tamil_words) - 1]
        expected_out = np.array(expected[1:])
        expected_in_ta  = np.array(expected[:-1])
        
        if len(expected_in.tolist()) < 1 or len(expected_in_ta.tolist()) < 1: 
            continue
        output = model(torch.from_numpy(expected_in).to(dev),torch.from_numpy(expected_in_ta).to(dev))        
        o = torch.squeeze(output,0)
        et = torch.from_numpy(expected_out).to(dev)
      
        if o.ndim == 1:
          o = o.unsqueeze(0)
          
        loss += criterion(o,et)
        epoch_loss += loss
        if d % 1000 == 0:
          print(d,epoch_loss/d)
        loss.backward()
        optimizer.step()
        d += 1
    print(epoch_loss/25500)
    if e % 1 == 0:
        torch.save({
                    'epoch': e,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': epoch_loss,
                    },"model/model_translation_transformer_without_withmask_{}.pth".format(e))
        print("saved")

using cuda:0
0.001
0.001
46
0 tensor(inf, grad_fn=<DivBackward0>)
1000 tensor(0.4070, grad_fn=<DivBackward0>)
2000 tensor(0.3981, grad_fn=<DivBackward0>)
3000 tensor(0.3862, grad_fn=<DivBackward0>)
4000 tensor(0.3790, grad_fn=<DivBackward0>)
5000 tensor(0.3745, grad_fn=<DivBackward0>)
6000 tensor(0.3670, grad_fn=<DivBackward0>)
7000 tensor(0.3674, grad_fn=<DivBackward0>)
8000 tensor(0.3621, grad_fn=<DivBackward0>)
9000 tensor(0.3622, grad_fn=<DivBackward0>)
10000 tensor(0.3578, grad_fn=<DivBackward0>)
11000 tensor(0.3570, grad_fn=<DivBackward0>)
12000 tensor(0.3544, grad_fn=<DivBackward0>)
13000 tensor(0.3544, grad_fn=<DivBackward0>)
14000 tensor(0.3533, grad_fn=<DivBackward0>)
15000 tensor(0.3520, grad_fn=<DivBackward0>)
16000 tensor(0.3502, grad_fn=<DivBackward0>)
17000 tensor(0.3487, grad_fn=<DivBackward0>)
18000 tensor(0.3492, grad_fn=<DivBackward0>)
19000 tensor(0.3472, grad_fn=<DivBackward0>)
20000 tensor(0.3460, grad_fn=<DivBackward0>)
21000 tensor(0.3447, grad_fn=<DivBackward0>

KeyboardInterrupt: ignored

In [ ]:
# if len(out_i) >= 4:
    #   tot_score_1 += bleu_score([output], [[e]],weights=(1, 0, 0, 0))
    #   div1 += 1
    # if len(out_i) >= 4:  
    #   tot_score_2 += bleu_score([output], [[e]],weights=(0.5, 0.5, 0, 0))
    #   div2 += 1
    # if len(out_i) >= 4:   
    #   tot_score_3 += bleu_score([output], [[e]],weights=(0.33, 0.33, 0.33, 0))
    #   div3 += 1
    # if len(out_i) >= 4:
    #   tot_score_4 += bleu_score([output], [[e]],weights=(0.25, 0.25, 0.25, 0.25))
    #   div4 += 1

    d += 1

# tot_score_1 += corpus_bleu(refs, candidates,weights=(1, 0, 0, 0))
# tot_score_2 += corpus_bleu(refs, candidates,weights=(0.5, 0.5, 0, 0))
# tot_score_3 += corpus_bleu(refs, candidates,weights=(0.33, 0.33, 0.33, 0))
# tot_score_4 += corpus_bleu(refs, candidates,weights=(0.25, 0.25, 0.25, 0.25))


# print(tot_score_1)
# print(tot_score_2)
# print(tot_score_3)
# print(tot_score_4)


# print(tot_score_1/div1)
# print(tot_score_2/div2)
# print(tot_score_3/div3)
# print(tot_score_4/div4)
# print(div1)
# print(div2)
# print(div3)
# print(div4)

In [ ]:
# Testing 

dev = torch.device("cpu")

model = Network()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.1)
 
checkpoint = torch.load("model_translation_transformer_without_withmask_49.pth",map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
 
model.eval()
tot_score_1 = 0
tot_score_2 = 0
tot_score_3 = 0
tot_score_4 = 0
start = 00
end = 1000
div1 = 0
div2 = 0
div3 = 0
div4 = 0
refs = []
candidates = []
div = 0
for d in range(start,end):
    print(d)
    
    expected = english_sentences_dev[d]
    
    # expected = [len(english_words) - 2] + [english_words.index(s.lower()) for s in expected]+ [len(english_words) - 1]
    # expected_in = np.array(expected[1:-1])
    
    e = [len(english_words) - 2]
    for s in expected:
      if s.lower() in english_words:
        e.append(english_words.index(s.lower()))
      else:
        e.append(len(english_words) - 2)
    e.append(len(english_words) - 1)
    expected_in = np.array(e[1:-1])

    expected = tamil_sentences_dev[d]
    print(expected)
    
    # expected = [len(tamil_words) - 2] + [tamil_words.index(s.lower()) for s in expected]+ [len(tamil_words) - 1]
    # expected_out = np.array(expected[1:])
    # exp = expected_out[:-1]
    expected_out = np.array([0,0])
    output,out_i = model(torch.from_numpy(expected_in).to(dev),torch.from_numpy(expected_out).to(dev))        
    print(output)
    print()
    

    if len(out_i) >= 4:
        tot_score_1 += sentence_bleu([expected], output,weights=(1, 0, 0, 0),smoothing_function=smoothie)
        tot_score_2 += sentence_bleu([expected], output,weights=(0.5, 0.5, 0, 0),smoothing_function=smoothie)
        tot_score_3 += sentence_bleu([expected], output,weights=(0.33, 0.33, 0.33, 0),smoothing_function=smoothie)
        tot_score_4 += sentence_bleu([expected], output,weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=smoothie)
        div += 1

    

print(div)
print(tot_score_1/div)
print(tot_score_2/div)
print(tot_score_3/div)
print(tot_score_4/div)

0
['SIS', 'மூன்று', '36', 'வயது', 'நடுத்தர', 'ஊதியம்', 'கடந்த', 'மூன்று', 'ஆண்டுகளில்']
['edmundsஇன்', 'பெற்றார்']

1
['நீங்கள்', 'தேர்வு', 'செய்ய', 'வேண்டும்', '']
['நீங்கள்', 'ஒரு', 'தேர்வு', 'வேண்டும்']

2
['பிட்ச்', 'நீங்கள்', 'நல்ல', 'உங்கள்', 'என்', 'என்', 'அலுவலக', 'outta']
['என்', 'அலுவலகத்தில்', 'உங்கள்', 'கழுதை']

3
['மற்றும்', 'உலக', 'எங்களை', 'பற்றி', 'மறக்க', 'முடியாது']
['மற்றும்', 'மறக்க', 'வேண்டும்']

4
['என்', 'செல்', 'போன்', 'எச்சரிக்கைகள்', 'மூலம்']
['என்', 'தொலைபேசி']

5
['இந்த', 'சாரணர்', 'கப்பல்', 'இருந்தது', 'வெற்றிடத்தை', 'செலுத்தப்படுகின்றன', 'ஆயிரக்கணக்கான', 'ஒன்றாகும்']
['இந்த', 'கப்பல்', 'ஒரு', 'இட']

6
['பெண்', 'பாடல்உள்ளதுஒரு', 'இருண்ட', 'மற்றும்', 'ஒரு', 'நான்', 'வாழ்க்கை', 'பதற்றமான', 'பக்க']
['பெண்', 'தன்மை', 'ஒரு']

7
['நேரங்களிலேயே', 'அவரது', 'அலுவலகத்தில்', 'அருகில்', 'சக', 'மூடிய', 'கதவுகளுக்குப்']
['வாடகை', 'மணிக்கு', 'உடன்']

8
['இருக்கின்றனஅவற்றில்', 'போராட', 'இன்னும்', 'ஆனால்', 'எனக்கு', 'தெரியாது', 'ஏன்']
['ஆனால்', 'நான்', 'இன்னும்', 'உள்ளது

In [ ]:
tot_score_1 += corpus_bleu(refs, candidates,weights=(1, 0, 0, 0))
tot_score_2 += corpus_bleu(refs, candidates,weights=(0.5, 0.5, 0, 0))
tot_score_3 += corpus_bleu(refs, candidates,weights=(0.33, 0.33, 0.33, 0))
tot_score_4 += corpus_bleu(refs, candidates,weights=(0.25, 0.25, 0.25, 0.25))


print(tot_score_1)
print(tot_score_2)
print(tot_score_3)
print(tot_score_4)

0
0
0
0


In [ ]:
# Training 
0.6247454936843057
0.558424484924263
0.5021806829604721
0.4098991335710353

# Validation - model emb100 withmask - 
0.2731762741090531
0.16716821471207702
0.12234304955446894
0.10730310606055712

In [ ]:
Some Good results:

GT:   ['NORAD', 'உறுதிப்படுத்துகிறது']
Pred: ['norad', 'உறுதிப்படுத்துகிறது']

GT:   ['நான்', 'சவாரி', 'தேவை']
Pred: ['நான்', 'சவாரி', 'தேவை']

GT:   ['இந்த', 'நாட்டின்', 'இருண்ட', 'மணி', 'ஒன்றாக']
Pred: ['இந்த', 'நாட்டின்', 'இருண்ட', 'மணி', 'ஒன்றாக']

GT:   ['எனவே', 'என்', 'புகைப்படத்தை', 'கிளிக்', 'செய்யவும்']
Pred: ['எனவே', 'என்', 'புகைப்படத்தை', 'கிளிக்', 'செய்யவும்']

Some Bad results:

GT:   ['ஆமாம்', 'ஆனால்', 'நான்', 'உள்ளே', 'போனாள்']
Pred: ['ஆமாம்', 'ஆனால்', 'நான்', 'உள்ளே', 'இல்லை']

GT:   ['அமெரிக்க', 'ஒரு', 'GO', 'கொடுக்க']
Pred: ['அமெரிக்க', 'ஒரு', 'ஏற்றவும்']



In [ ]:
Final viva testing
0.13415528096883744
0.061247798736327665
0.0401099444892067
0.03235098449999804